In [ ]:
# LLM 모델을 통한 연산 예제
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)
# llm = ChatOpenAI(model_name='gpt-4o',temperature=0)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12."

llm.invoke(prompt).content

In [ ]:
# LLM 모델을 통한 연산 예제
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0)

# 덧셈을 수행하는 함수 정의
def plus(a, b): 
    return a + b

# 에이전트 생성
agent = initialize_agent(
    # 사용할 LLM 모델
    llm=llm,

    # 추론 과정 디버깅(에전트가 하는 작업의 모든 과정 및 추론을 직접 확인해볼 수 있음)
    verbose = True, 

    # 에이전트 유형 설정
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,

    # 에이전트가 사용할 도구 모음(Tools)
    tools=[
        StructuredTool.from_function(
            func = plus, # 위에서 정의해놓은 덧셈 함수를 도구 모음에 추가
            name = "Sum Calculator", # 도구의 이름을 "덧셈 계산기"로 정의 
            description="Use this to perform sums of two numbers. This tool take two arguments, both should be numbers. till finish." # 도구에 대한 설명 작성
        )
    ]
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12."


# 이번엔 에이전트에게 프롬프트를 입력(위에선 LLM모델에 입력했음)
agent.invoke(prompt)

In [8]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1)


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Apple Company'}`


Discover real-time Apple Inc. Common Stock (AAPL) stock prices, quotes, historical data, news, and Insights for informed trading and investment decisions. Stay ahead with Nasdaq. Historical Stock Price Lookup. Investor Relations > Stock Price . Apple Footer Stock analysis for Apple Inc (AAPL:NASDAQ GS) including stock price, stock chart, company news, key statistics, fundamentals and company profile. A high-level overview of Apple Inc. (AAPL) stock. Stay up to date on the latest stock price, chart, news, analysis, fundamentals, trading and investment tools. Get a real-time Apple Inc (AAPL) stock price quote with breaking news, financials, statistics, charts and more. Get a real-time Apple Inc (AAPL) stock price quote with breaking news, financials, statistics, charts and more. ... joins 'Squawk Box' to discuss the latest market trends, where inves

{'input': "Give me information on Apple's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have.",
 'output': "The stock symbol for Apple Inc. is AAPL. \n\nApple Inc. (AAPL) is listed on the NASDAQ GS stock exchange. Here are some key details about Apple's stock:\n- Real-time stock price: [AAPL Stock Price](https://www.nasdaq.com/market-activity/stocks/aapl)\n- Historical data, news, and insights for informed trading and investment decisions are available on Nasdaq.\n\nTo analyze if Apple's stock is a potential good investment, we can consider factors such as:\n1. Financial performance: Review Apple's revenue growth, profitability, and cash flow.\n2. Market trends: Analyze the industry trends and competitive landscape.\n3. Stock valuation: Evaluate the stock's price-to-earnings ratio and other valuation metrics.\n4. Future prospects: Consider upcoming product launches, innovations, and market expansion plans.\n\nWould you like to de